In [1]:
import memory_profiler
from metro.ita import *
from metro import multiplex as mx
import cProfile
import os

In [2]:
multi = mx.read_multi()
multi.read_od(layer = 'taz', key = 'taz', od_file = '1_data/taz_od/0_1.txt', sep = " ")
g, od = multi.to_igraph()

In [3]:
n_nodes = 50
sub_od = {key : od[key] for key in od.keys()[:n_nodes]}

In [4]:
def summary_constructor(es, base_cost):
    def get_path_summary(path):
        edges = eval(path)
        congested_time_m = sum([es[e]['congested_time_m'] for e in edges])
        uniform_time_m = sum([es[e]['uniform_time_m'] for e in edges])
        free_flow_time_m = sum([es[e]['free_flow_time_m'] for e in edges])
        dist_km = sum([es[e]['dist_km'] for e in edges])
        base = sum([es[e][base_cost] for e in edges])
        weighted_capacity = (1.0 * sum([es[e]['dist_km']*es[e]['capacity'] for e in edges]))
        weighted_flow = (1.0 * sum([es[e]['dist_km']*es[e]['flow'] for e in edges]))
        gamma = [weighted_flow / weighted_capacity if weighted_capacity > 0 else np.nan][0]
        gradient = (1.0 * sum([es[e]['gradient'] for e in edges]))
        return (congested_time_m, uniform_time_m, free_flow_time_m, dist_km, base, gamma, gradient)
    return get_path_summary

def make_details_df(df, es, base_cost):
    get_path_summary = summary_constructor(es, base_cost)
    df['congested_time_m'], df['uniform_time_m'], df['free_flow_time_m'], df['dist_km'], df['base_cost'], df['gamma'], df['gradient'] = zip(*df['path'].map(get_path_summary))
    del df['path']
    return df[['o', 'd', 'p', 'flow', 'dist_km', 'uniform_time_m', 'free_flow_time_m', 'congested_time_m', 'base_cost', 'gamma', 'gradient']]

In [5]:
def ITA(g, od, base_cost = 'free_flow_time_m', P = [0.4, 0.3, 0.2, 0.1], a = 0.15, b = 4., scale = .25, details = False):
    
    import time 
    

    columns = ['o', 'd', 'p', 'flow', 'path']
    flow_dict = defaultdict(int)

    es = g.es
    
    es['flow'] = 0
    es['congested_time_m'] = list(es[base_cost])
    
    j = 0
    for p in P: 
        start = time.clock()
        paths_list = pd.DataFrame(columns = columns)
        for o in od:
            ds = od[o]
            if len(ds) > 0:
                targets = ds.keys()
                paths = g.get_shortest_paths(o, 
                                             to=targets, 
                                             weights='congested_time_m', 
                                             mode='OUT', 
                                             output="epath")

                # Update flow dict
                for i in range(len(targets)):
                        flow = od[o][targets[i]]
                        for e in paths[i]:
                            flow_dict[e] += p * scale * flow


                # Update paths list
                if details:
                    update_piece = [{'o' : o, 
                    'd' : targets[i], 
                    'p' : p,
                    'flow' : scale * od[o][targets[i]], 
                    'path' : str(paths[i])} for i in range(len(targets))]
                    update_piece = pd.DataFrame(update_piece)
                    paths_list = paths_list.append(update_piece)
                

        # Assign the flows to the graph
        for key in flow_dict:
            es[key]['flow'] = flow_dict[key]
            es[key]['congested_time_m'] = BPR(base = es[key][base_cost], 
                                    flow = es[key]['flow'], 
                                    capacity = float(es[key]['capacity']),
                                    a = a,
                                    b = b)
        if details:
            paths_list.to_csv('3_throughput/paths_list_' + str(j) + '.csv')
            j += 1
            del paths_list
        print 'assignment for p = ' + str(p) + ' completed in ' + str(round((time.clock() - start) / 60.0, 1)) + 'm'
        

    compute_gradient('free_flow_time_m', 'flow', 'capacity', a, b, es)
    
    # Compute details
    if details:
        df = pd.DataFrame(columns = columns)
        for k in range(len(P)):
            df = df.append(pd.read_csv('3_throughput/paths_list_' + str(k) + '.csv'))
            os.remove('3_throughput/paths_list_' + str(k) + '.csv')
        df = make_details_df(df, es, base_cost)
        df = agg_df(df)
        con_map = { v.index : v['con_name'] for v in g.vs}
        df['o_con'] = df.o.map(con_map.get)
        df['d_con'] = df.d.map(con_map.get)
        nx_map = { v.index : v['name'] for v in g.vs}
        df['o_nx'] = df.o.map(nx_map.get)
        df['d_nx'] = df.d.map(nx_map.get)
        del df['o']
        del df['d']
        return df

In [6]:
df = ITA(g, od, details = True) 

assignment for p = 0.4 completed in 16.7m
assignment for p = 0.3 completed in 32.9m


KeyboardInterrupt: 

In [ ]:
df.describe()